In [33]:
import skimage
from skimage import io
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
import os
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib

In [34]:
data_dir = os.path.join('data_WiSAR','data')
data_dir = os.path.join('data')

In [35]:
def preprocess_image(img, mask, homography):
    '''
    cuts out provided mask for timestamp
    warps image to the perspective of the center camera with provided homography matrix
    '''
    img = img[..., :3].astype(np.float32)/255
    img[mask==0] = np.nan
    processed_img = img.copy()
    for dim in range(3):
        processed_img[:,:,dim] = cv2.warpPerspective(img[:,:,dim],homography,img.shape[:2],borderMode = cv2.BORDER_CONSTANT,borderValue = np.nan) 
    return processed_img

In [36]:
def integrate_images(img_list):
    '''
    averages all images in img_list
    removes pixels of the average image that are not covered by all images
    '''
    integrated_img = np.nanmean(np.array(img_list), axis = 0)
    integrated_img_nan = integrated_img.copy()
    for dim in range(3):
        integrated_img_nan[:,:,dim] = np.where(np.isnan(img_list).any(axis = 0).all(axis = 2), np.nan, integrated_img[:,:,dim])
    return integrated_img_nan

In [37]:
def crop_image(img):
    '''
    crops images to a hardcoded size (315x700), so that no nan-values occur in any of the images
    '''
    cropped_img = img[370:685,140:840,:]
    return cropped_img

In [38]:
def label_image(img,label,shift = False,max_pixel = 1,dim=3):
    '''
    labels image with provided labels as red rectangles
    shift: set True for cropped images and False for images in original size
    '''
    if shift:
        x_shift, y_shift = 140, 370
    else:
        x_shift, y_shift = 0, 0
    for lab in label:
        x = lab[0]-x_shift
        y = lab[1]-y_shift
        w = lab[2]
        h = lab[3]
        if dim == 3:
            for i in range(x, x+w):
                img[y,i,:] = [max_pixel]*3
                img[y+h,i,:] = [max_pixel]*3
            for j in range(y, y+h):
                img[j,x,:] = [max_pixel]*3
                img[j,x+w,:] = [max_pixel]*3
        elif dim == 2:
            for i in range(x, x+w):
                img[y,i] = max_pixel
                img[y+h,i] = max_pixel
            for j in range(y, y+h):
                img[j,x] = max_pixel
                img[j,x+w] = max_pixel
    return img

In [39]:
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

In [40]:
def process_data(dataset, save = False, plot = True, label = False, only_cropped = False):
    '''
    creates integrated and cropped images as well as images showing the variance between 7 timesteps 
    dataset: 'train', 'test' or 'validation'
    save: set to True if images should be saved in subfolders of 'data_WiSAR/data'
    plot: set to True if images should be plotted in the jupyter notebook
    label: set to True if images should be labelled given that labels are provided - labelled images get
        saved in a separate folder
    only_cropped: Save only the cropped images
    '''

    mask = skimage.io.imread(os.path.join(data_dir,'mask.png'))
    if label:
        try:
            with open(os.path.join(data_dir,dataset,'labels.json')) as f:
                labels = json.load(f)
        except:
            label = False
    labelled_flag = ''
    if label:
        labelled_flag = '_labelled'
    if save:
        if not os.path.isdir(os.path.join(data_dir,'processed_images_'+dataset+labelled_flag)):
            os.makedirs(os.path.join(data_dir,'processed_images_'+dataset+labelled_flag))
    for folder in os.listdir(os.path.join(data_dir, dataset)):
        if folder.startswith(dataset) or folder.startswith('valid'):
            if save:
                save_folder = os.path.join(data_dir,'processed_images_'+dataset+labelled_flag,folder)
                if not os.path.isdir(save_folder):
                    os.makedirs(save_folder)
            with open(os.path.join(data_dir,dataset,folder,'homographies.json')) as f:
                homographies = json.load(f)
            if label:
                label_coords = labels[folder]
            all_processed_images = []
            cropped_images = []
            for timepoint in [0,1,2,3,4,5,6]:
                processed_images = []
                for camera in ['B01', 'B02', 'B03', 'B04', 'B05', 'G01', 'G02', 'G03', 'G04', 'G05']:
                    image = skimage.io.imread(os.path.join(data_dir,dataset,folder,str(timepoint)+'-'+camera+'.png'))
                    homography = np.array(homographies[str(timepoint) + '-' + camera])
                    processed_image = preprocess_image(image, mask, homography)
                    processed_images.append(processed_image)
                    all_processed_images.append(processed_image)
                integrated_image = integrate_images(processed_images)
                cropped_image = crop_image(integrated_image)
                cropped_images.append(cropped_image)
                if label:
                    integrated_image = label_image(integrated_image, label = label_coords)
                    cropped_image = label_image(cropped_image, label = label_coords, shift = True)

                if save and not only_cropped:
                    matplotlib.image.imsave(os.path.join(save_folder,'integrated_image_'+str(timepoint)+'.png'), integrated_image)
                    matplotlib.image.imsave(os.path.join(save_folder,'cropped_image_'+str(timepoint)+'.png'), cropped_image)

                if plot:
                    plt.figure()
                    plt.imshow(integrated_image)
                    plt.figure()
                    plt.imshow(cropped_image)            

            integrated_image_all = integrate_images(all_processed_images)
            cropped_image_all = crop_image(integrated_image_all)
            var_image = np.nanvar(cropped_images, axis = 0)/np.max(np.nanvar(cropped_images, axis = 0))
                      
            if label:
                integrated_image_all = label_image(integrated_image_all, label = label_coords)
                cropped_image_all = label_image(cropped_image_all, label = label_coords, shift = True)
                var_image = label_image(var_image, label = label_coords, shift = True)

            if save and not only_cropped:
                matplotlib.image.imsave(os.path.join(save_folder,'integrated_image_all.png'), integrated_image_all)
                matplotlib.image.imsave(os.path.join(save_folder,'cropped_image_all.png'), cropped_image_all)
                matplotlib.image.imsave(os.path.join(save_folder,'image_var.png'), var_image)
            if save and only_cropped:
                matplotlib.image.imsave(os.path.join(save_folder,'cropped_image_all.png'), cropped_image_all)

            if plot:
                plt.figure()
                plt.imshow(integrated_image_all)
                plt.figure()
                plt.imshow(cropped_image_all)   
                plt.figure()
                plt.imshow(var_image)
                plt.figure()
                plt.imshow(var_threshold_image) 

In [41]:
def create_cropped_dataset(dataset):
    '''
    saves all cropped images from process_data() in one folder per dataset
    dataset: 'train', 'test', 'validation', 'validation_labelled'
    requires process_data(dataset, save = False, plot = True, label = False) to be run first
    '''
    save_path = os.path.join(data_dir,'cropped_dataset',dataset)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    for folder in os.listdir(os.path.join(data_dir, 'processed_images_' + dataset)):
        for timepoint in [0,1,2,3,4,5,6]:
            cropped_image = skimage.io.imread(os.path.join(data_dir,'processed_images_' + dataset,folder,'cropped_image_'+str(timepoint)+'.png'))
            cropped_image = cropped_image[..., :3].astype(np.float32)/255
            matplotlib.image.imsave(os.path.join(save_path,folder+'_'+'cropped_'+str(timepoint)+'.png'), cropped_image)

In [42]:
def create_variance_dataset(dataset):
    '''
    saves all variance images from process_data() in one folder per dataset
    dataset: 'train', 'test', 'validation', 'validation_labelled'
    requires process_data(dataset, save = False, plot = True, label = False) to be run first
    '''
    save_path = os.path.join(data_dir,'variance_dataset',dataset)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    for folder in os.listdir(os.path.join(data_dir, 'processed_images_' + dataset)):
        var_image = skimage.io.imread(os.path.join(data_dir,'processed_images_' + dataset,folder,'image_var.png'))
        var_image = var_image[..., :3].astype(np.float32)/255
        matplotlib.image.imsave(os.path.join(save_path,folder+'_variance.png'), var_image)

In [43]:
def create_variance_channel_dataset(dataset, colour):
    '''
    saves the selected colour channel of the variance images from create_variance_dataset() as greyscale image 
    as well as the subtraction of the green channel from the selected channel in two different folders per dataset
    dataset: 'train', 'test', 'validation', 'validation_labelled'
    colour: 'blue', 'red'
    requires create_variance_dataset(dataset, colour) to be run first
    '''
    save_path_colour = os.path.join(data_dir,'variance_dataset_'+colour,dataset)
    if not os.path.isdir(save_path_colour):
        os.makedirs(save_path_colour)
    save_path_colour_green = os.path.join(data_dir,'variance_dataset_'+colour+'-green',dataset)
    if not os.path.isdir(save_path_colour_green):
        os.makedirs(save_path_colour_green)
    if dataset == 'validation_labelled':
        with open(os.path.join(data_dir,'validation','labels.json')) as f:
            labels = json.load(f)
    if colour == 'red':
        channel = 0
    elif colour == 'blue':
        channel = 2
    for folder in os.listdir(os.path.join(data_dir, 'processed_images_' + dataset)):
                
        var_image = skimage.io.imread(os.path.join(data_dir,'variance_dataset',dataset,folder+'_variance.png'))
        var_image = var_image[..., :3].astype(np.float32)/255
        channel_image = var_image[...,channel]
        matplotlib.image.imsave(os.path.join(save_path_colour,folder+'_variance_'+colour+'.png'), channel_image, cmap='gray')

        green_image = var_image[...,1]
        green_sub_image = channel_image - green_image
        green_sub_image = np.where(green_sub_image<0, 0, green_sub_image)
        green_sub_image /= np.max(green_sub_image)
        if dataset == 'validation_labelled':
            label_coords = labels[folder]
            green_sub_image = label_image(green_sub_image, label = label_coords, shift = True, dim = 2)

        matplotlib.image.imsave(os.path.join(save_path_colour_green,folder+'_variance_'+colour+'-green.png'), green_sub_image, cmap='gray')
        

In [44]:
def create_variance_green_dataset(dataset):
    '''
    saves the green channel of the variance images from create_variance_dataset() as greyscale images 
    not relevant for analysis but included for the pupose of completeness (used in powerpoint presentation)
    dataset: 'train', 'test', 'validation', 'validation_labelled'
    requires create_variance_dataset(dataset, colour) to be run first
    '''
    save_path_colour = os.path.join(data_dir,'variance_dataset_green',dataset)
    if not os.path.isdir(save_path_colour):
        os.makedirs(save_path_colour)
    if dataset == 'validation_labelled':
        with open(os.path.join(data_dir,'validation','labels.json')) as f:
            labels = json.load(f)

    for folder in os.listdir(os.path.join(data_dir, 'processed_images_' + dataset)):
                
        var_image = skimage.io.imread(os.path.join(data_dir,'variance_dataset',dataset,folder+'_variance.png'))
        var_image = var_image[..., :3].astype(np.float32)/255
        channel_image = var_image[...,1]
        matplotlib.image.imsave(os.path.join(save_path_colour,folder+'_variance_green.png'), channel_image, cmap='gray')


In [45]:
def create_contrast_brightness_dataset(dataset,colour,contrast=96,brightness=32):
    '''
    adjusts contrast and brightness on the green subtracted create_variance_channel_dataset()
    and saves them in a folder per dataset 
    dataset: 'train', 'test', 'validation', 'validation_labelled'
    colour: 'blue', 'red'
    contrast: int between -128 and 128, default: 96
    brightness: int between -128 and 128, default: 32
    requires create_variance_channel_dataset(dataset, colour) to be run first
    '''
    save_path = os.path.join(data_dir,'variance_dataset_'+colour+'-green_adjusted',dataset)
    if dataset == 'validation_labelled':
        with open(os.path.join(data_dir,'validation','labels.json')) as f:
            labels = json.load(f)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)
    for folder in os.listdir(os.path.join(data_dir, 'processed_images_' + dataset)):
        if dataset == 'validation_labelled':
            label_coords = labels[folder]

        green_sub_image = skimage.io.imread(os.path.join(data_dir,'variance_dataset_'+colour+'-green',dataset,folder+'_variance_'+colour+'-green.png'))
        green_sub_image = green_sub_image[..., :3]

        adjusted_image = apply_brightness_contrast(green_sub_image, brightness, contrast)
        if dataset == 'validation_labelled':
            adjusted_image = label_image(adjusted_image, label = label_coords, shift = True, max_pixel = 255)
        matplotlib.image.imsave(os.path.join(save_path,folder+'_variance_'+colour+'-green_c'+str(contrast)+'_b'+str(brightness)+'.png'), adjusted_image, cmap='gray')

In [31]:
# run all

process_data('train', save = True, plot = False, label = False, only_cropped=False)
process_data('test', save = True, plot = False, label = False, only_cropped=False)
process_data('validation', save = True, plot = False, label = False, only_cropped=False)
process_data('validation', save = True, plot = False, label = True, only_cropped=False)

for dataset in ['train', 'test', 'validation', 'validation_labelled']:
    create_cropped_dataset(dataset)
    create_variance_dataset(dataset)
#     create_variance_green_dataset(dataset)
    for colour in ['blue', 'red']:
        create_variance_channel_dataset(dataset, colour)
        create_contrast_brightness_dataset(dataset, colour)

/var/folders/g8/7dsc7c7d5757b7m6xbqnwy680000gn/T/ipykernel_52366/1819771478.py:6: RuntimeWarning: Mean of empty slice
  integrated_img = np.nanmean(np.array(img_list), axis = 0)


In [47]:
process_data('train', save = True, plot = False, label = False, only_cropped=True)
process_data('test', save = True, plot = False, label = False, only_cropped=True)
process_data('validation', save = True, plot = False, label = False, only_cropped=True)

/var/folders/g8/7dsc7c7d5757b7m6xbqnwy680000gn/T/ipykernel_52366/1819771478.py:6: RuntimeWarning: Mean of empty slice
  integrated_img = np.nanmean(np.array(img_list), axis = 0)
